## CSUS - CSc 177-02 Data Warehousing and Data Mining - Project 1: Clustering  
### 2016 U.S. presidential election Twitter analysis  

**Group members: Aaron Enberg, Nima Sarrafzadeh, Kyne Liu**  
**Professor: Haiquan (Victor) Chen**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import (
    preprocessing,  
    cluster as sk_cluster,
    metrics as metrics
)
from sklearn.feature_selection import VarianceThreshold
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import (
    cross_val_score,
    GridSearchCV
)
import sklearn.feature_extraction.text as sk_text

%matplotlib inline
pd.set_option('display.max_colwidth', -1)

In [2]:
column_names = ['name', 'screen_name', 'user_id', 
                'followers_count', 'friends_count', 
                'location', 'description', 'created_at', 
                'status_id', 'language', 'place', 
                'retweet_count', 'favorite_count', 'text']

tweets = pd.read_table('data/clinton_trump_tweets.txt', names=column_names, encoding='ISO-8859-1')

In [3]:
print(tweets.shape)
print(tweets.dtypes)

(5250980, 14)
name               object
screen_name        object
user_id            int64 
followers_count    int64 
friends_count      int64 
location           object
description        object
created_at         object
status_id          int64 
language           object
place              object
retweet_count      int64 
favorite_count     int64 
text               object
dtype: object


## Preprocessing

In [4]:
tweets.drop(['name', 'screen_name',   
            'description', 'created_at',
            'status_id', 'language', 
            'place', 'location'], axis=1, inplace=True)
tweets.head()

,user_id,followers_count,friends_count,retweet_count,favorite_count,text
0,1519696717,132,263,0,1,@NWAJimmy I've read it now though brother. Was pretty spot on Lots of bright spots but a lot to work on. Exactly as an exhibition should be!
1,109945090,2154,2034,1937,0,RT @wikileaks: New poll puts Pirate Party on course to win Iceland's national elections on Saturday. https://t.co/edTqjeJaQ6
2,1450086582,797,1188,0,0,@gaystoner821 I think New Orleans spoiled me with food. I need to try and branch out in BR.
3,167177185,204,448,891,0,RT @LOLGOP: ACA needs fixes but know da facts: *70% can get covered in marketplaces for under $75/month *Hikes affect 3% *GOP will uninsu
4,1191022351,775,154,7,0,RT @FastCompany: Alphabet shares soar on better-than-expected earnings as mobile video strategy pays off https://t.co/bokbXngMJt https://t.


### Remove all retweets

In [5]:
# lines that start with "RT" followed by a space
pattern = r'^RT\s'
 
# matches retweets and removes them
tweets = tweets[tweets.text.str.match(pattern) == False]

In [6]:
tweets.shape

(2416818, 6)

### Extract all handles and hashtags

In [7]:
# match all hashtags and handles in a tweet, ignoring possible email addresses
pattern = r'(?<=^|(?<=[^a-zA-Z0-9-\.]))@([A-Za-z_]+[A-Za-z0-9_]+)|(?<=^|(?<=[^a-zA-Z0-9-\.]))#([A-Za-z_]+[A-Za-z0-9_]+)'

""" returns a DataFrame with a MultiIndex:
    First index is our original index. Second index is "match" which is a running
    total of the number of occurences of hashtags and mentions for a particular 
    tweet. """
handles_hashtags = tweets.text.str.extractall(pattern)

# make the dataframe look nice 
handles_hashtags.columns = ['handles', 'hashtags']
handles_hashtags = handles_hashtags.reset_index().set_index('level_0')
del handles_hashtags.index.name
handles_hashtags.drop(['match'], axis=1, inplace=True)

# stack handles and hashtags into one column
handles_hashtags = pd.concat([handles_hashtags.handles, handles_hashtags.hashtags]).dropna().to_frame(name='handles_hashtags')

tweets = tweets.join(handles_hashtags, how='inner').reset_index()
tweets.drop(['text'], axis=1, inplace=True)

### keep active users with at least 20 distinct hashtags/handles

In [8]:
# returns users along with the distinct hashtags/handles they've used
handles_hashtags_distinct = tweets.groupby('user_id')['handles_hashtags'].unique().to_frame()
# retrieve only those who have used 20 or more distinct handles/hashtags
users_active = handles_hashtags_distinct[handles_hashtags_distinct.handles_hashtags.str.len() > 19]

In [9]:
''' returns users along with all the hashtags and handles (including duplicates) they've used 
    (each occurrence of a hashtag/handle will also show up in the list) '''
handles_hashtags_all = tweets.reset_index().groupby('user_id')['handles_hashtags'].apply(list).to_frame()

users_active = users_active.join(handles_hashtags_all, lsuffix='_distinct', rsuffix='_all', how='inner')

In [10]:
# construct a dataframe with totals for retweets, favorites, followers, and friends for each user
tweets.drop(['handles_hashtags', 'index'], axis=1, inplace=True)
tweets.set_index('user_id', inplace=True)
count_retweets_favorites = tweets.groupby(tweets.index)['retweet_count', 'favorite_count'].sum()
tweets.drop(['retweet_count', 'favorite_count'], axis=1, inplace=True)
count_followers_friends = tweets[~tweets.index.duplicated(keep='first')]
users_active_stats = count_retweets_favorites.join(count_followers_friends, how='inner')
users_active_stats.drop(['favorite_count', 'followers_count', 'friends_count'], axis=1, inplace=True)
users_active_stats.head()


,retweet_count
user_id,
150,0
1437,0
1512,0
1644,6
1668,0


In [11]:
# convert each list of handles/hashtags into a string
users_active['handles_hashtags_stringified'] = users_active['handles_hashtags_all'].apply(lambda x: ' '.join(map(str, x)))

# list of strings, each containing all of the handles and hashtags for a user
corpus = list(users_active['handles_hashtags_stringified'])

### hashtags/handles that have been used by at least 20 distinct users (min_df=20)

In [12]:
# constructs a dataframe with only the handles/hashtags used by 20 users 
vectorizer = sk_text.CountVectorizer(min_df=20, lowercase=True, encoding='ISO-8859-1')
matrix = vectorizer.fit_transform(corpus)
df_idx_id = pd.DataFrame(matrix.toarray(), index=users_active.index, columns=vectorizer.get_feature_names())

In [13]:
# append retweet/favorite counts and friend/follower counts to end of each record
df_idx_id = df_idx_id.join(users_active_stats, how='inner')

# rows are users, columns are hashtags/handles
# values are frequency of a handle/hashtag
df_idx_id.head()

,_altright_anew,_carja,_cfj_,_makada_,_proud_american,_realvalentina_,a_miller48,abbydphillip,abbymartin,abbymartinm,...,zaibatsunews,zaidjilani,zekejmiller,zerohedge,zhaabowekwe,zigmanfreud,zika,zimmermanrob,zip90210,retweet_count
user_id,,,,,,,,,,,,,,,,,,,,,
1644,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6
1737,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
2391,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,18
2426,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,199
14763,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9


In [14]:
# represent each handle/hashtag as an integer
df_idx_hashtags = df_idx_id.transpose().reset_index(drop=True).transpose()

In [15]:
labels = pd.read_table('./data/clinton_trump_user_classes.txt', 
                       names=['user_id', 'label'])
labels.set_index('user_id', inplace=True)
df_idx_hashtags = labels.join(df_idx_hashtags, how='inner')
y = df_idx_hashtags['label']
df_idx_hashtags.drop('label', axis=1, inplace=True)

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_idx_hashtags, 
                                                    y, test_size=0.2, 
                                                    random_state=42)

In [17]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(13652, 4036)
(3414, 4036)
(13652,)
(3414,)


In [18]:
std_scaler = preprocessing.StandardScaler()
X_train_std = std_scaler.fit_transform(X_train)
X_test_std = std_scaler.fit_transform(X_test)

#### SVM Classification

In [28]:
svm_clf = SVC(C=1, kernel='rbf', degree=3, 
              gamma='auto', coef0=0.0, shrinking=True, 
              probability=False, tol=0.001, cache_size=200, 
              class_weight=None, verbose=False, max_iter=-1, 
              decision_function_shape='ovr', random_state=42)
svm_clf.fit(X_train_std, y_train)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=42, shrinking=True,
  tol=0.001, verbose=False)

In [29]:
y_pred = svm_clf.predict(X_test_std)
print(metrics.f1_score(y_test, y_pred))

0.7768281101614435


#### Decision Tree Classification

In [26]:
tree_clf = DecisionTreeClassifier(criterion='gini', 
                                  splitter='best', 
                                  max_depth=None, 
                                  min_samples_split=2, 
                                  min_samples_leaf=1, 
                                  min_weight_fraction_leaf=0.0, 
                                  max_features=None, 
                                  random_state=42, 
                                  max_leaf_nodes=99, 
                                  min_impurity_decrease=0.0, 
                                  min_impurity_split=None, 
                                  class_weight=None, 
                                  presort=False)
tree_clf.fit(X_train_std, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=99,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

In [27]:
y_pred = tree_clf.predict(X_test_std)
print(metrics.f1_score(y_test, y_pred))

0.7604364326375712


#### kNN Classification with grid search for best k and 5-fold cross-validation

In [19]:
params = {'n_neighbors': list(range(1, 20))}
knn_grid_search_cv = GridSearchCV(KNeighborsClassifier(), params, cv=5, n_jobs=-1, verbose=1)

knn_grid_search_cv.fit(X_train_std, y_train)

Fitting 5 folds for each of 19 candidates, totalling 95 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed: 79.2min
[Parallel(n_jobs=-1)]: Done  95 out of  95 | elapsed: 245.9min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [20]:
knn_grid_search_cv.best_estimator_

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=12, p=2,
           weights='uniform')

In [21]:
knn_grid_search_cv.best_score_

0.6596103135071785

In [22]:
y_pred = knn_grid_search_cv.best_estimator_.predict(X_test_std)
print(metrics.f1_score(y_test, y_pred))

0.7226118500604595


In [24]:
knn_grid_search_cv.cv_results_

/home/aaron/.virtualenvs/CSc177/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/aaron/.virtualenvs/CSc177/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/aaron/.virtualenvs/CSc177/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/

{'mean_fit_time': array([6.45201759, 5.97357082, 5.49793668, 4.92522311, 5.34484549,
        5.38179779, 5.24446721, 5.0155261 , 5.0378087 , 4.92368097,
        5.05490537, 5.05236354, 5.01986804, 5.19169617, 5.0062984 ,
        5.04905152, 4.91266131, 4.90150666, 5.12212958]),
 'mean_score_time': array([144.96333842, 189.55677805, 209.5252883 , 217.93238144,
        232.22594905, 245.38913784, 252.83323212, 257.58317728,
        261.59472752, 265.31801805, 271.72247858, 274.07894468,
        274.71545439, 276.571071  , 277.0685307 , 277.74630294,
        278.8623404 , 282.13796101, 273.6343677 ]),
 'mean_test_score': array([0.61610021, 0.59749487, 0.64305596, 0.63397304, 0.65470261,
        0.64767067, 0.65484911, 0.65199238, 0.65792558, 0.65719309,
        0.65865807, 0.65961031, 0.65902432, 0.65777908, 0.6552886 ,
        0.65433636, 0.65089364, 0.65265163, 0.64825667]),
 'mean_train_score': array([0.98632067, 0.82255341, 0.80870942, 0.78288893, 0.76796433,
        0.76193954, 0.747